In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
from imdb import IMDb

In [2]:
# Get data frame with data from movies
df = pd.read_csv('western.csv', sep=';')

In [3]:
df.head(10)

Unnamed: 0                   title  \
0           0               Wild Bill   
1           1              Wyatt Earp   
2           2               Bad Girls   
3           3              The Outlaw   
4           4             The Shooter   
5           5  Terror in a Texas Town   
6           6              Pale Rider   
7           7           Alvarez Kelly   
8           8     High Plains Drifter   
9           9  Rough Night in Jericho   

                                                plot  
0  The early career of legendary lawman Wild Bill...  
1  Wyatt Earp is a movie about a man and his fami...  
2  Four prostitutes join together to travel the O...  
3  Western legends Pat Garrett, Doc Holliday and ...  
4  The story is of a small town in the early west...  
5  A Swedish whaler is out for revenge when he fi...  
6  A mysterious preacher protects a humble prospe...  
7  During the Civil War, Mexican cattleman Alvare...  
8  A gunfighting stranger comes to the small sett...  
9  A former deputy and a strong-willed widow dete...

In [4]:
es = Elasticsearch()

In [5]:
es.indices.create(index="movie-index", ignore=400)

{u'error': {u'index': u'movie-index',
  u'index_uuid': u'_NYpkmbySyG5mrNq45rz1w',
  u'reason': u'index [movie-index/_NYpkmbySyG5mrNq45rz1w] already exists',
  u'root_cause': [{u'index': u'movie-index',
    u'index_uuid': u'_NYpkmbySyG5mrNq45rz1w',
    u'reason': u'index [movie-index/_NYpkmbySyG5mrNq45rz1w] already exists',
    u'type': u'index_already_exists_exception'}],
  u'type': u'index_already_exists_exception'},
 u'status': 400}

In [6]:
for i in range(1, len(df.index)):
    es.index(index='movie-index', doc_type='movie', body={
        'id': i,
        'title': df.loc[i]['title'],
        'plot': df.loc[i]['plot']
    })

In [7]:
res = es.search(index='movie-index', body={"query": {"fuzzy" : { "title" : "texs"}}}, size=3)

In [8]:
res['hits']['hits']

[{u'_id': u'AVy3OYxNUTIIhy1g5VSV',
  u'_index': u'movie-index',
  u'_score': 2.5036051,
  u'_source': {u'id': 5,
   u'plot': u'A Swedish whaler is out for revenge when he finds out that a greedy oil man murdered his father for their land.',
   u'title': u'Terror in a Texas Town'},
  u'_type': u'movie'},
 {u'_id': u'AVy24QSgUTIIhy1g5UAN',
  u'_index': u'movie-index',
  u'_score': 2.4423828,
  u'_source': {u'id': 5,
   u'plot': u'A Swedish whaler is out for revenge when he finds out that a greedy oil man murdered his father for their land.',
   u'title': u'Terror in a Texas Town'},
  u'_type': u'movie'},
 {u'_id': u'AVy2_tfMUTIIhy1g5UJt',
  u'_index': u'movie-index',
  u'_score': 2.4253943,
  u'_source': {u'id': 5,
   u'plot': u'A Swedish whaler is out for revenge when he finds out that a greedy oil man murdered his father for their land.',
   u'title': u'Terror in a Texas Town'},
  u'_type': u'movie'}]

In [9]:
for doc in res['hits']['hits']:
    print doc['_source']['plot']

A Swedish whaler is out for revenge when he finds out that a greedy oil man murdered his father for their land.
A Swedish whaler is out for revenge when he finds out that a greedy oil man murdered his father for their land.
A Swedish whaler is out for revenge when he finds out that a greedy oil man murdered his father for their land.


In [10]:
# Delete index when too many values are indexed
#es.indices.delete(index='movie-index', ignore=[400, 404])
#es.indices.delete(index='western', ignore=[400, 404])

In [11]:
movies = es.search(
            index='western',
            body={"query": {
                    "multi_match": {
                        "fields":  [ "title", "plot" ],
                        "query": "ranger",
                        "fuzziness": "AUTO"
                    }
                },
                "highlight": {
                    "fields": {
                        "title": {"number_of_fragments": 0},
                        "plot": {"number_of_fragments": 0}
                    }
                }
            },
            size=3
        )

In [12]:
movies['hits']['hits']

[{u'_id': u'AVy3N0GzUTIIhy1g5VPT',
  u'_index': u'western',
  u'_score': 3.2168498,
  u'_source': {u'id': 12,
   u'plot': u'Edited version of the 1938 Republic serial "The Lone Ranger."',
   u'title': u'Hi-Yo Silver'},
  u'_type': u'movie',
  u'highlight': {u'plot': [u'Edited version of the 1938 Republic serial "The Lone <em>Ranger</em>."']}},
 {u'_id': u'AVy3N0YvUTIIhy1g5VPs',
  u'_index': u'western',
  u'_score': 3.0563602,
  u'_source': {u'id': 37,
   u'plot': u'The sole surviving Texas Ranger of an ambush arranged by Butch Cavandish returns to fight back as a masked hero.',
   u'title': u'The Legend of the Lone Ranger'},
  u'_type': u'movie',
  u'highlight': {u'plot': [u'The sole surviving Texas <em>Ranger</em> of an ambush arranged by Butch Cavandish returns to fight back as a masked hero.'],
   u'title': [u'The Legend of the Lone <em>Ranger</em>']}},
 {u'_id': u'AVy3N1MmUTIIhy1g5VQz',
  u'_index': u'western',
  u'_score': 2.9207013,
  u'_source': {u'id': 108,
   u'plot': u'John M

In [23]:
movies['hits']['hits'][0]['highlight']['plot'][0]

u'Edited version of the 1938 Republic serial "The Lone <em>Ranger</em>."'

In [25]:
movies['hits']['total']

26